In [15]:
import pandas as pd
import os
import re
import warnings
import json
import unicodedata
import string

warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.2f}'.format

In [16]:
def ajustes(x):
    x = str(x).strip()
    return 0 if x == "-" or x == "R$ -" else x

def strip(x):
    # Converter para string e remover espaços iniciais e finais
    x = str(x).strip()

    # Remover acentos
    x = ''.join(
        c for c in unicodedata.normalize('NFKD', x)
        if not unicodedata.combining(c)
    )

    # Remover pontuações
    x = x.translate(str.maketrans('', '', string.punctuation))
    x = str(x).upper()

    return x

In [18]:
df_cotacoes = pd.read_excel('D:\Codigos\CEASA-Dados\Migração dados Ceasa Online\historico-cotacoes.xlsx', decimal=',')
df_cotacoes[['Maximo','Mcomum','Minimo']] = df_cotacoes[['Maximo','Mcomum','Minimo']].applymap(ajustes)
df_cotacoes['Unidade'] = df_cotacoes['Unidade'].apply(strip)
df_cotacoes[['Produto','Unidade']] = df_cotacoes[['Produto','Unidade']].astype('category')
df_cotacoes[['Maximo','Mcomum','Minimo']] = df_cotacoes[['Maximo','Mcomum','Minimo']].astype('float')
df_cotacoes

,Cotacao,Produto,Unidade,Maximo,Mcomum,Minimo
0,2024-10-02,00000-FOLHAGENS VARIADAS,UND,30.00,20.00,20.00
1,2024-10-01,00000-FOLHAGENS VARIADAS,UND,30.00,20.00,20.00
2,2024-10-03,00000-FOLHAGENS VARIADAS,UND,30.00,20.00,20.00
3,2024-10-04,00000-FOLHAGENS VARIADAS,UND,30.00,20.00,20.00
4,2024-10-07,00000-FOLHAGENS VARIADAS,UND,30.00,20.00,20.00
...,...,...,...,...,...,...
126164,2024-11-22,ZAMIOCULCA,UND,35.00,35.00,35.00
126165,2024-12-26,ZAMIOCULCA,UND,35.00,35.00,35.00
126166,2024-12-27,ZAMIOCULCA,UND,35.00,35.00,35.00
126167,2024-12-29,ZAMIOCULCA,UND,35.00,35.00,35.00


In [19]:
with open('D:\Codigos\CEASA-Dados\Migração dados Ceasa Online\produtos.json', 'r', encoding='utf-8') as arquivo:
    arquivo_json = json.load(arquivo)

nao_consta = []

lista_produtos_cotacao = df_cotacoes['Produto'].unique()

for elemento in lista_produtos_cotacao:
    if elemento.lower() not in [key.lower() for key in arquivo_json.keys()]:
        nao_consta.append(elemento)

nao_consta.sort()

if not nao_consta:  # Verifica se a lista está vazia
    print('Não há nada que não conste no arquivo JSON. Tudo está em ordem!')
else:
    print(f'{len(nao_consta)} itens não constam no arquivo JSON:')
    for item in nao_consta:
        print(item)

Não há nada que não conste no arquivo JSON. Tudo está em ordem!


In [20]:
# Copiar o DataFrame original
morango = df_cotacoes.copy(deep=True)

# Filtrar apenas os produtos que são "MORANGO"
morango = morango[morango['Produto'] == 'MORANGO']

# Garantir que a coluna 'Cotacao' está no formato datetime
morango['Cotacao'] = pd.to_datetime(morango['Cotacao'], errors='coerce')

# Criar colunas de ano e mês
morango['ANO'] = morango['Cotacao'].dt.year
morango['MES'] = morango['Cotacao'].dt.month

# Verificar e tratar valores NaN na coluna 'Mcomum'
morango['Mcomum'] = pd.to_numeric(morango['Mcomum'], errors='coerce')  # Garantir que é numérico
morango = morango.dropna(subset=['Mcomum'])  # Remover linhas onde 'Mcomum' seja NaN

# Agrupar por Produto, ANO e MES e calcular a média de 'Mcomum'
morango = morango.groupby(['ANO', 'MES'])['Mcomum'].mean().reset_index()

# Exibir o resultado
morango.sort_values(by=['ANO','MES'])

,ANO,MES,Mcomum
0,2020,1,9.52
1,2020,2,11.45
2,2020,3,0.00
3,2020,4,0.00
4,2020,5,0.00
5,2020,6,0.00
6,2020,7,0.00
7,2020,8,0.00
8,2020,9,0.00
9,2020,10,0.00


## Ajustando os nomes dos Produtos

In [37]:
with open('D:\Codigos\CEASA-Dados\Migração dados Ceasa Online\produtos.json', 'r', encoding='utf-8') as arquivo:
    arquivo_json = json.load(arquivo)

def mapear_produtos(produto):
    if pd.isnull(produto):
        return produto
    return arquivo_json.get(str(produto).strip().upper(), produto)



In [38]:
df_cotacoes['Produto'] = df_cotacoes['Produto'].apply(mapear_produtos)
df_cotacoes

,Cotacao,Produto,Unidade,Maximo,Mcomum,Minimo
0,2021-01-03,Abacate Comum,KG,9.00,9.00,8.00
1,2021-01-03,Abacaxi Caiena (Hawai),UND,5.00,5.00,3.00
2,2021-01-03,Abacaxi Perola,UND,4.50,4.50,3.00
3,2021-01-03,Abacaxi Terra de Areia,UND,3.00,2.25,1.20
4,2021-01-03,Abobora,KG,4.00,4.00,2.50
...,...,...,...,...,...,...
127792,2023-12-28,Verdes Eucaliptus,MOL,35.00,35.00,15.00
127793,2023-12-28,Verdes Ruscus,MOL,25.00,24.00,15.00
127794,2023-12-28,Verdes (Samambaia Preta),MOL,15.00,15.00,15.00
127795,2023-12-28,Violeta,UND,5.90,5.90,5.00


# Formatação dos dados para Importação

## Cotação

In [43]:
df_cotacao_marco = pd.read_excel('D:\Codigos\CEASA-Dados\Migração dados Ceasa Online\ceasa_online_cotacao_mensal_marco_2025.xlsx', decimal=',')
df_cotacao_marco.columns = df_cotacao_marco.columns.str.strip()
df_cotacao_marco['unidade'] = df_cotacao_marco['unidade'].map(lambda x: x.lower().strip() if isinstance(x, str) else x)
df_cotacao_marco['produtovariedade_id'] = df_cotacao_marco['produtovariedade_id'].apply(mapear_produtos)
df_cotacao_marco['preco_moda_kg'] = df_cotacao_marco['preco_moda_kg'].apply(lambda x: round(x,2))

# df_cotacao_marco.to_csv(r'C:\Users\divtec-dados\Desktop\cotação mensal.csv', sep=',', index=False, encoding='utf-8')
# df_cotacao_marco['produtovariedade_id'].loc[df_cotacao_marco['produtovariedade_id'].duplicated()]
df_cotacao_marco

,Id,unidade,mes,ano,data_geracao,data_fechamento,produtovariedade_id,preco_moda_kg,ativa
0,NaN,kg,3,2025,2025-03-26,2025-03-26,Abacate Avocado,23.75,True
1,NaN,kg,3,2025,2025-03-26,2025-03-26,Abacate Comum,5.12,True
2,NaN,und,3,2025,2025-03-26,2025-03-26,Abacaxi Caiena (Hawai),10.00,True
3,NaN,und,3,2025,2025-03-26,2025-03-26,Abacaxi Perola,7.94,True
4,NaN,und,3,2025,2025-03-26,2025-03-26,Abacaxi Terra de Areia,4.84,True
...,...,...,...,...,...,...,...,...,...
228,NaN,mol,3,2025,2025-03-26,2025-03-26,Verdes Ruscus,18.38,True
229,NaN,und,3,2025,2025-03-26,2025-03-26,Violeta,6.00,True
230,NaN,cx,3,2025,2025-03-26,2025-03-26,Violeta Mini,45.00,True
231,NaN,kg,3,2025,2025-03-26,2025-03-26,Yacon,10.38,True


In [45]:
df_cotacao_referencia = pd.read_excel('D:\Codigos\CEASA-Dados\Migração dados Ceasa Online\ceasa_online_cotacao_referencia.xlsx', decimal=',')
df_cotacao_referencia.columns = df_cotacao_referencia.columns.str.strip()
df_cotacao_referencia['unidade'] = df_cotacao_referencia['unidade'].map(lambda x: x.lower().strip() if isinstance(x, str) else x)
df_cotacao_referencia['produtovariedade_id'] = df_cotacao_referencia['produtovariedade_id'].apply(mapear_produtos)
df_cotacao_referencia[['maior_preco_kg','media_preco_kg','menor_preco_kg','moda_preco_kg']] = df_cotacao_referencia[['maior_preco_kg','media_preco_kg','menor_preco_kg','moda_preco_kg']] .apply(lambda x: round(x,2))

# df_cotacao_marco.to_csv(r'C:\Users\divtec-dados\Desktop\cotação mensal.csv', sep=',', index=False, encoding='utf-8')
df_cotacao_referencia.to_csv(r'C:\Users\divtec-dados\Desktop\cotação referencial.csv', sep=',', index=False, encoding='utf-8')

# df_cotacao_marco['produtovariedade_id'].loc[df_cotacao_marco['produtovariedade_id'].duplicated()]
df_cotacao_referencia

,Id,unidade,ordem,data_atualizacao,produtovariedade_id,maior_preco_kg,media_preco_kg,menor_preco_kg,moda_preco_kg,ativa
0,NaN,kg,NaN,2025-03-25,Abobora,3.50,3.00,2.00,3.00,True
1,NaN,kg,NaN,2025-03-25,Abobrinha Italiana,3.33,3.00,2.00,3.00,True
2,NaN,kg,NaN,2025-03-25,Abobrinha Tronco,4.00,3.00,2.33,3.00,True
3,NaN,dz,NaN,2025-03-25,Acelga,50.00,40.00,35.00,40.00,True
4,NaN,dz,NaN,2025-03-25,Agrião,20.00,20.00,20.00,20.00,True
...,...,...,...,...,...,...,...,...,...,...
238,NaN,cx,NaN,2025-03-25,Ovo Branco,250.00,240.00,240.00,240.00,True
239,NaN,cx,NaN,2025-03-25,Ovo Caipira,360.00,360.00,300.00,360.00,True
240,NaN,cxt,NaN,2025-03-25,Ovo Cordorna,7.50,6.50,6.50,6.50,True
241,NaN,cx,NaN,2025-03-25,Ovo Vermelho,300.00,290.00,280.00,290.00,True


## Notas Fiscais

# Solicitação do Léo

In [ ]:
datas = ['2025-03-25','2024-03-26','2023-03-28','2022-03-29','2021-03-30']
produtos = ['Tomate Longa Vida','Cebola','Banana Prata / Branca','Alface Crespa','Agrião','Caqui Mole','Batata Branca','Mandioca / Aipim','Melancia','Abacaxi Perola','Brocolis','Couve Flor','Morango']
df = df_cotacoes.copy(deep=True)
df = df.loc[(df['Cotacao'].isin(datas)) & df['Produto'].isin(produtos)].reset_index()

df = df.pivot_table(index='Produto', values='Mcomum', columns='Cotacao').reset_index()
df = df.sort_values(by=['Produto'])
df = df.fillna(0)
df.columns = ['Produto','2021','2022','2023','2024','2025']
df['2021 X 2025'] = (((df['2025'] - df['2021']) / df['2025']) * 100).apply(lambda x: f"{x:.2f}%")
df['2022 X 2025'] = (((df['2025'] - df['2022']) / df['2025']) * 100).apply(lambda x: f"{x:.2f}%")
df['2023 X 2025'] = (((df['2025'] - df['2023']) / df['2025']) * 100).apply(lambda x: f"{x:.2f}%")
df['2024 X 2025'] = (((df['2025'] - df['2024']) / df['2025']) * 100).apply(lambda x: f"{x:.2f}%")
df.to_excel('Cotações Selecionadas.xlsx', index=False)